# **Chemostat: Logistic + Clogging**
by: Edwin Saavedra C.

In [1]:
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as wd
import sympy as sp
import pandas as pd
from IPython.display import display
from scipy.integrate import odeint
from scipy.optimize import fsolve,root

In [2]:
## MATPLOTLIB rcParams
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'medium'
plt.rcParams['axes.labelpad'] = 5.0
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['axes.edgecolor'] = 'gray'

In [3]:
def stream(Y,t,p):
    x,s = Y
    dxdt =  x*(1-x)*s/(p[0]+s) - p[1]*x
    dsdt =  p[2]*np.power(np.maximum(1-x,0),p[4])*(1-s) - p[3]*x*(1-x)*s/(p[0]+s)
    return [dxdt,dsdt]

def rates(Y,t,p):
    x,s = Y
    x,s = np.array(x),np.array(s)
    dqdt =  p[2]*np.power(np.maximum(1-x,0),p[4])*df['Value']['Y']*df['Value']['q']
    #drdt =  p[3]*(1-x)*s/(p[0]+s)*df['Value']['q']*df['Value']['Y']*df['Value']['S0']
    drdt =  p[3]*x*(1-x)*s/(p[0]+s)*df['Value']['q']*df['Value']['Y']*df['Value']['S0']
    return [dqdt,drdt]

In [4]:
df = pd.DataFrame()
df["Index"]  = ['Y','q','Ks','b','Q/V','S0','n']
df.set_index("Index",inplace=True)
df["Label"]  = [r'Y',r'\hat{q}',r'K_s',r'b',r'Q/V',r'S^0',r'n']
df["Value"]  = [0.42,10,20,0.15,1000/2000,50,1]
df["Units"]  = ["mg(X)/mg(S)","mg(S)/mg(X)·d","mg(S)/L","1/d","1/d","mg(S)/L","-"]
df["Slider"] = [wd.FloatText(value=v,description=f'${k}$',step=0.1) for k,v in zip(df["Label"],df["Value"])]

nd = pd.DataFrame()
nd["Index"]  = np.arange(0,5,1)
nd.set_index("Index",inplace=True)
nd["Label"]  = [fr"\pi_{{{s}}}" for s in nd.index]
nd["Value"]  = np.arange(10,15,1)
nd["Slider"] = [wd.FloatText(value="{:.4f}".format(v),description=f'${k}$') for k,v in zip(nd["Label"],nd["Value"])]

In [5]:
def update_values():
    df["Value"] = [w.value for w in df["Slider"]]
    nd["Value"] = [w.value for w in nd["Slider"]]

def calculate_ps(change):   
    nd["Slider"][0].value = df["Slider"]['Ks'].value/df["Slider"]['S0'].value
    nd["Slider"][1].value = df["Slider"]['b'].value/(df["Slider"]['q'].value*df["Slider"]['Y'].value)
    nd["Slider"][2].value = df["Slider"]['Q/V'].value/(df["Slider"]['q'].value*df["Slider"]['Y'].value)
    nd["Slider"][3].value = 1.0
    nd["Slider"][4].value = df["Slider"]['n'].value
    
    update_values()

In [6]:
for slider in df["Slider"]:
    slider.observe(calculate_ps,'value')

calculate_ps(True)

In [7]:
whitespace = wd.HTML("&nbsp;&nbsp;<b>&#8594;</b>&nbsp;&nbsp;")
layout = wd.Layout(justify_content='center',align_items='center')

eqTex0 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dX}{dt} &=& Y\hat{q}X\left(1-\dfrac{X}{X_{\rm max}}\right)\dfrac{S}{K_S + S} - bX \\
    \dfrac{dS}{dt} &=& \tfrac{Q}{V}\left(1-\dfrac{X}{X_{\rm max}}\right)^n (S_\infty-S) - \hat{q}X\left(1-\dfrac{X}{X_{\rm max}}\right)\dfrac{S}{K_S + S}
\end{array}
\end{equation}''',layout=layout)

eqTex1_1 = wd.HTMLMath(r'''\begin{equation*}
\begin{array}{rcl}
    s &=& S/S_\infty \\
    x &=& X/X_{\rm max} \\
    \tau &=& \hat{q}Yt \\
\end{array}
\end{equation*}''')

eqTex1_2 = wd.HTMLMath(r'''\begin{equation*}
\begin{array}{rcl}
    \pi_0 &=& K_S/S_\infty \\
    \pi_1 &=& b/\hat{q}Y \\
    \pi_2 &=& Q/V\hat{q}Y \\
    \pi_3 &=& X_{\rm max}/S_\infty Y \\
\end{array}
\end{equation*}''')

eqTex1 = wd.HBox([eqTex1_1,whitespace,eqTex1_2],layout=layout)

eqTex2 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dx}{d\tau} &=& x(1-x)\dfrac{s}{\pi_0 + s} - \pi_1 x \\
    \dfrac{ds}{d\tau} &=& \pi_2 \left(1-x\right)^n \left(1-s\right) - \pi_3 x(1-x)\dfrac{s}{\pi_0 + s}\\
\end{array}
\end{equation}''',layout=layout)

eqTex3 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{cc}
    x=0 & s=1 \\
    x = \mathsf{:S} & s = \mathsf{>.<}
\end{array}
\end{equation}''',layout=layout)

eqTexs = [eqTex0,eqTex1,eqTex2,eqTex3]

In [8]:
kw_streams = {'density':1,'linewidth':1.5,'arrowsize':0.8,
              'arrowstyle':'->','color':[1,1,1,0.5],'minlength':0.2}

output = wd.Output(layout=layout)

@output.capture(clear_output=True)
def plotStream():
    update_values()
    
    time = np.arange(0,50,0.5)
    xlin = np.linspace(0,1,61)
    slin = np.linspace(0,1,61)
    x,s  = np.meshgrid(xlin,slin)
    p = nd["Value"]
    initVals = [v.value for v in init]
    
    ## Calculate vector field
    dx,ds = stream([x,s],None,p)

    ## Calculate trayectories given initial condition
    tray = odeint(stream,initVals,time,args=(p,))
    
    ## Calculate steady-state
    #ssx,sss = fsolve(stream, initVals,args=(_,p))
    sol = root(stream,[tray[-1,0],tray[-1,1]],args=(None,p))
    ssx,sss = sol.x
    
    ## Calculate independent rates
    rate = rates(tray.T,None,p)
    
    
    kw_streams = {'density':2,'linewidth':1.5,'arrowsize':0.8,
              'arrowstyle':'->','color':'k','minlength':0.1}

    fig = plt.figure(figsize=[12,6])
    gs = gridspec.GridSpec(6, 12)
    ax = fig.add_subplot(gs[:,:6])
    ax.set_aspect("equal")
    ax.streamplot(x,s,dx,ds,**kw_streams,zorder=2)
    ax.scatter([0,ssx],[1,sss],s=250,clip_on=False,ec='k',label="Steady-states",zorder=3)
    ax.scatter(tray[0,0],tray[0,1],s=250,marker='X',clip_on=False,ec='k',
               label=r"$(x_0,s_0)$",zorder=4,c='wheat')
    
    if ssx < 0.0001:
        ax.annotate("Washout",
                  xy=(0.0, 0.99), xycoords='data',
                  xytext=(0.5, 0.5), textcoords='data',
                  size=20, va="center", ha="center",
                  bbox=dict(boxstyle="round4", fc="pink", ec="red"),
                  arrowprops=dict(arrowstyle="-|>",
                                  connectionstyle="arc3,rad=-0.2",
                                  fc="w",ec='red'))
    
    ax.plot(tray.T[0],tray.T[1],lw=5,c='purple')

    ax.grid(True,ls='dashed',lw=1,c=[0.5,0.5,0.5,0.5])
    ax.set_xlabel("$x$ [-]",)
    ax.set_ylabel("$s$ [-]")
    ax.set(xlim=[0,1],ylim=[0,1])
    ax.legend(loc='upper right',fontsize=10)

    pax = fig.add_subplot(gs[:3,7:])
    pax.set_facecolor("#F5DEB343")
    for variable in tray.T:
        pax.plot(time,variable,lw=3)
    pax.set_ylim([0,1])
    pax.set_xlim(left=0)
    pax.set_ylabel(r'Non-dim concentration',fontsize=8)
    pax.axes.get_xaxis().set_visible(False)
    pax.legend([r'$x(\tau)$',r'$s(\tau)$'],fontsize=10)

    rax = fig.add_subplot(gs[3:,7:])
    rax.sharex(pax)
    rax.set_facecolor("white")
    rax2 = rax.twinx()
    
    p1, = rax.plot(time,rate[0],lw=3,label=r'$\frac{Q}{V}(\tau) \: \mathsf{[m^3/d]}$',c='b')
    p2, = rax2.plot(time,rate[1],lw=3,label=r'$r_{\rm ut}(\tau) \: \mathsf{[mg/d/m^3]}$',c='r')
    rax.legend(handles=[p1,p2],fontsize=10)
    
    rax.set_xlim(left=0)
    rax.set_ylim(bottom=0)
    rax2.set_ylim(bottom=0)
                 
    rax.set_xlabel(r'$\tau$')
    rax.set_ylabel(r'Dilution rate $Q/V$',fontsize=8)
    rax2.set_ylabel(r'Rate substrate util. $r_{\rm ut}$',fontsize=8)
    plt.show();
  
    return None

def updateFig(b):
    plotStream();

In [9]:
x_init = wd.FloatSlider(value=0.01,min=0,max=1.0,step=0.01,description=r"$x_0$")
s_init = wd.FloatSlider(value=0.99,min=0,max=1.0,step=0.01,description=r"$s_0$")
init = [x_init,s_init]

updateFig(True);

In [10]:
plotButton = wd.Button(description="Plot!",icon='drafting-compass')
plotButton.on_click(updateFig)

gs = wd.GridspecLayout(12,2)
gs[0:5,0] = wd.VBox(list(df["Slider"]),layout=wd.Layout(align_items='stretch'))
gs[0:4,1] = wd.VBox(list(nd["Slider"])+[x_init,s_init])
gs[4,1] = plotButton
gs[5:,:] = wd.HBox([output])

accordion = wd.Accordion(eqTexs,layout=wd.Layout(align_items='stretch'))
accordion.selected_index = None
accordion.set_title(0,'Governing ODEs')
accordion.set_title(1,'Nondimensionalization')
accordion.set_title(2,'Non-dimensional ODEs')
accordion.set_title(3,'Steady-state solution')

mainBox = wd.VBox([accordion,gs])
display(mainBox)